<a href="https://colab.research.google.com/github/Yussefayman/MachineLearningFromScratch/blob/main/RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
num_classes = 10
num_epochs = 2
batch_size = 100
lr = 0.001
input_size = 28
seq_len = 28
hidden_size = 128
num_layers = 2

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset = torchvision.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())

In [11]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [12]:
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super(RNN,self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.rnn = nn.RNN(input_size, hidden_size, num_layers,batch_first = True)
    self.fc = nn.Linear(hidden_size,num_classes)

  def forward(self,x):
    h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
    # out: (batch_size,seq_length,hidden_size)
    # out: (100,28,128)
    out,_ = self.rnn(x,h0)
    #Decode the hidden state of the last time step
    out = out[:,-1,:]
    out = self.fc(out)
    return out

In [13]:
model = RNN(input_size,hidden_size,num_layers,num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)


In [15]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images = images.reshape(-1,seq_len,input_size).to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
      print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/2], Step [100/600], Loss: 1.0151
Epoch [1/2], Step [200/600], Loss: 0.6546
Epoch [1/2], Step [300/600], Loss: 0.4936
Epoch [1/2], Step [400/600], Loss: 0.4252
Epoch [1/2], Step [500/600], Loss: 0.4596
Epoch [1/2], Step [600/600], Loss: 0.3460
Epoch [2/2], Step [100/600], Loss: 0.2359
Epoch [2/2], Step [200/600], Loss: 0.2031
Epoch [2/2], Step [300/600], Loss: 0.2234
Epoch [2/2], Step [400/600], Loss: 0.2992
Epoch [2/2], Step [500/600], Loss: 0.3383
Epoch [2/2], Step [600/600], Loss: 0.2125


In [16]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images,labels in test_loader:
    images = images.reshape(-1,seq_len,input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data,1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 93.48 %
